# SIMULATED DATASET 

In [24]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import torch
from crossevopred.src.model.dummy_model import DummyModel
from crossevopred.src.model.dummy_trainer import DummyTrainer
from crossevopred.utils.data_processing_utils import *
from crossevopred.src.data.dataset import ExpressionDataset
from crossevopred.src.data.encoder import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
def window_array(arr, window_size):
    return np.array([np.mean(arr[i:i+window_size]) for i in range(0, len(arr)-window_size+1, window_size)])

def write_bed_file(sequences, scores, output_bed):
    with open(output_bed, "w") as f:
        for i in range(len(sequences)):
            sequence = sequences[i]
            score = scores[i]
            f.write("chr1\t{}\t{}\t{}\t{}\n".format(i, i+len(sequence), ','.join([str(x) for x in score]), sequence))
# Simulate N sequences of length L. Each A +4, C+3, G+2, T+0 . 
# The output is a bed file with chr1 x y fasta_sequence [score_pos1, score_pos2, ...]
# in the simulation I want to have sequences that are very A rich and sequences that are very T rich.
# but still have some C and G.
def simulate_sequences(N, L, window_size, output_bed):
    sequences = []
    for i in range(int(N/2)):
        sequence = np.random.choice(["A", "C", "G", "T"], L, p=[0.5, 0.25, 0.25, 0])
        sequences.append("".join(sequence))
    for i in range(int(N/2)):
        sequence = np.random.choice(["A", "C", "G", "T"], L, p=[0, 0.25, 0.25, 0.5])
        sequences.append("".join(sequence))
    sequences = np.array(sequences)
    # scores are 4 for A, 3 for C, 2 for G, 0 for T
    scores = np.array([[4 if c == "A" else 3 if c == "C" else 2 if c == "G" else 0 for c in sequence] for sequence in sequences])
    scores = np.array([window_array(score, window_size) for score in scores])
    
    # write it in 3 different bed files (train, val, test)
    # proportions 80% train, 10% val, 10% test
    train_proportion = 0.8
    val_proportion = 0.1
    test_proportion = 0.1
    train_size = int(N * train_proportion)
    val_size = int(N * val_proportion)
    test_size = int(N * test_proportion)
    train_sequences = sequences[:train_size]
    val_sequences = sequences[train_size:train_size+val_size]
    test_sequences = sequences[train_size+val_size:]
    train_scores = scores[:train_size]
    val_scores = scores[train_size:train_size+val_size]
    test_scores = scores[train_size+val_size:]
    # write bed files
    write_bed_file(train_sequences, train_scores, output_bed + "_train.bed")
    write_bed_file(val_sequences, val_scores, output_bed + "_val.bed")
    write_bed_file(test_sequences, test_scores, output_bed + "_test.bed")

    return output_bed + "_train.bed", output_bed + "_val.bed", output_bed + "_test.bed"


In [95]:
# Create simulated data
training_file = "../simulated/simulated"
config_file = "../bin/crossevopred/config/test_config.yaml"
simulate_sequences(N = 100, L = 1000, window_size= 128, output_bed=training_file)


('../simulated/simulated_train.bed',
 '../simulated/simulated_val.bed',
 '../simulated/simulated_test.bed')

In [96]:
# Create simulated dataset
training_dataset = ExpressionDataset(training_file+ "_train.bed")
training_dataset_file = training_file+ "_test_dataset.pt"
torch.save(training_dataset, training_dataset_file)

# Create simulated dataset
test_dataset = ExpressionDataset(training_file+ "_test.bed")
test_dataset_file = training_file+ "_test_dataset.pt"
torch.save(test_dataset, test_dataset_file)

# Create simulated dataset
val_dataset = ExpressionDataset(training_file+ "_val.bed")
val_dataset_file = training_file+ "_val_dataset.pt"
torch.save(val_dataset, val_dataset_file)

In [97]:
trainining_datatset_loader.sequences.shape

torch.Size([100, 4, 1000])

In [98]:
# retrain model 
import pandas as pd
import numpy as np
from Bio import SeqIO
import torch
from crossevopred.src.model.dummy_model import DummyModel
from crossevopred.src.model.dummy_trainer import DummyTrainer
from crossevopred.utils.data_processing_utils import *
from crossevopred.src.data.dataset import ExpressionDataset
from crossevopred.src.data.encoder import *

model   = DummyModel()
trainer = DummyTrainer()
trainining_datatset_loader = torch.load(training_dataset_file)
# Train the model
trainer.train(trainining_datatset_loader, config_file=config_file, model=model)


forwarding...
forwarding...
forwarding...
forwarding...
forwarding...
forwarding...
forwarding...
forwarding...


KeyboardInterrupt: 

In [43]:
# predict on my test set
test_dataset_loader = torch.load(test_dataset_file)
encoder = DNAEncoder()

In [49]:
test_dataset_loader.sequences[0].shape

torch.Size([4, 1000])

In [44]:
with torch.no_grad():
        for sequence, label in test_dataset_loader:
                decoded_sequence = encoder.decode_one_hot(sequence.numpy())
                # count the number of A, C, G, T in string
                print("A: ", decoded_sequence.count("A"))
                print("C: ", decoded_sequence.count("C"))
                print("G: ", decoded_sequence.count("G"))
                print("T: ", decoded_sequence.count("T"))
                print(decoded_sequence)
                print(label)
                print(model(sequence))
                print("--------------------")

A:  0
C:  244
G:  254
T:  502
TTTTTTTCCTTTTTTCTGGTCCCCCTTTGTCGGGTCTGTGGGTCTTGGCGTCTTGTTTGGCTTTGTTTTGGTGGTTTGGCCCCTTTCTTTGCTGTGTGTTGTCTTTTGTTCGGGTTTTTCGTTTCTTCGGCGCTTTGCTCGGTTTGGTGGTTCCTCTCTTTTTCTCTGCTCCTGTTTTTGTTCGCCCTCTTCTTTTCCTTTGTCTGGTTTTTGGGCCTTTCCTGCTCCTGTTTGTTCCCCGTGCGTTTTTGTGTTTGTCGGGCGGTTTCTTGCTCCGTCTTCTTGTTTTCCTGTTCTGTGTTCTTTTTTCTCCGGTTGTCTTTTTTCTTGTTTCGCCTTCTTGTTCCTTCTGCCTGTGTTGGGTTCTCGTCTTCGCTTTGCTCGCCCCGTGTTGCGTGTTCTTTGTTTTCTCTTCGTCTCTTTGCCGTGGTCTTTTGGTGGCTCTGGCGTTCTTTTTTCGCTTCCTGTCCGGCTTTTGTCTCTTTTGTTGTGGTCTTGGGGCCGTGTCTTGCTTCCCTTTCGTGCCCTGGGCTCTTGCCTGCTGTTTTGTGTGCTGTTTTCCCTCGTGCTGTCTGTCTTTGCGCCCGCTGTGGGCCGTTGGTCTGTGGGGTCTTTTCTTCGTGGCCTCCTTTCTTGGTTTCGCTTTGCGTTTTTGCGTGTTCCTCTCCTGGCTTTGTCTTGTCTGTTGTTGTTCGTCTTTCTTCTTTGCCCTGGTGTCCTCGGTCTTCTTTGGTCTTGGGCCTTTCCCTGTGTTGGCCCTTGTGTTTTTTGCTCGGTTTGTTTTTCGTTTTCTTGTTCTTTGCCTTCCCCGTTCCTCGTTGGGTTGTTCTTCTCGGTCTTGTTTCCGGTGGTCGTCTCGGCCTTGTGGTTTTCTGCTTTTTCCGTTGGGGTTGCTTTGGCCTTCTGCTCCTTTGGTGGTGTTGTGCGCGTGTGTTCTCCTTTGTCTCGTGGTTTTTTGCCTCGTCGTCTG

In [45]:
torch.randn(1, 3, 10) 

tensor([[[ 0.3213, -1.9513,  0.2648, -0.7327,  0.0843,  1.3971, -1.1471,
          -0.0124,  0.7253, -0.6719],
         [-0.1158, -0.3666, -0.0502,  0.8536,  1.7015, -0.8840, -0.2631,
          -0.0226, -0.7647, -0.1910],
         [ 0.1111,  0.9971, -0.4549, -0.2262,  0.3083, -1.0041, -0.1656,
           0.1007, -1.4479,  0.9049]]])

In [36]:
torch.tensor([10.0])

tensor([10.])